 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [75]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [76]:
# Reading whale returns
whale_returns = pd.read_csv(".\Resources\whale_returns.csv", index_col = "Date", parse_dates = True, infer_datetime_format = True)
whale_returns.sample(3)

,SOROS FUND MANAGEMENT LLC,PAULSON & CO.INC.,TIGER GLOBAL MANAGEMENT LLC,BERKSHIRE HATHAWAY INC
Date,,,,
2017-11-14,-0.008979,-0.003418,-0.005497,-0.014813
2018-08-29,0.003578,0.002776,0.006234,0.001879
2018-02-15,0.022236,0.007007,0.010304,0.013290


In [77]:
# Count nulls
whale_returns.isnull().sum()

SOROS FUND MANAGEMENT LLC      1
PAULSON & CO.INC.              1
TIGER GLOBAL MANAGEMENT LLC    1
BERKSHIRE HATHAWAY INC         1
dtype: int64

In [78]:
# Drop nulls
whale_returns = whale_returns.dropna().copy()

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [79]:
# Reading algorithmic returns
algo_returns = pd.read_csv("./Resources/algo_returns.csv", index_col = "Date", parse_dates = True, infer_datetime_format = True)
algo_returns.sample(3)

,Algo 1,Algo 2
Date,,
2014-08-05,-0.002206,-0.003920
2015-05-21,0.006866,-0.000162
2014-11-17,-0.004657,-0.006085


In [80]:
# Count nulls
algo_returns.isnull().sum()

Algo 1    0
Algo 2    6
dtype: int64

In [81]:
# Drop nulls
algo_returns = algo_returns.dropna().copy

## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [82]:
# Reading S&P 500 Closing Prices
sp500_prices = pd.read_csv(".\Resources\sp500_history.csv", index_col = "Date", parse_dates = True, infer_datetime_format = True)
sp500_prices.sample(3)

,Close
Date,
2015-12-14,$2021.94
2015-07-27,$2067.64
2014-06-16,$1937.78


In [83]:
# Check Data Types
sp500_prices.dtypes

Close    object
dtype: object

In [84]:
# Fix Data Types
sp500_prices["Close"] = sp500_prices["Close"].str.replace("$", "", regex = False).astype("float")

In [85]:
# Calculate Daily Returns
sp500_returns = sp500_prices.pct_change()

In [86]:
# Drop nulls
sp500_returns = sp500_returns.dropna().copy()

In [119]:
# Rename `Close` Column to be specific to this portfolio.
sp500_returns = sp500_returns.rename(columns = {"Date": "S&P500"}).copy()
sp500_returns.sample(3)

,Close
Date,
2015-09-17,0.016430
2013-08-09,0.001154
2016-12-29,0.004659


## Combine Whale, Algorithmic, and S&P 500 Returns

In [88]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.


---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [89]:
# Plot daily returns of all portfolios


#### Calculate and Plot cumulative returns.

In [90]:
# Calculate cumulative returns of all portfolios

# Plot cumulative returns


---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [91]:
# Box plot to visually show risk


### Calculate Standard Deviations

In [92]:
# Calculate the daily standard deviations of all portfolios


### Determine which portfolios are riskier than the S&P 500

In [93]:
# Calculate  the daily standard deviation of S&P 500

# Determine which portfolios are riskier than the S&P 500


### Calculate the Annualized Standard Deviation

In [94]:
# Calculate the annualized standard deviation (252 trading days)


---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [95]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window

# Plot the rolling standard deviation


### Calculate and plot the correlation

In [96]:
# Calculate the correlation

# Display de correlation matrix


### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [97]:
# Calculate covariance of a single portfolio

# Calculate variance of S&P 500

# Computing beta

# Plot beta trend


## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [98]:
# Use `ewm` to calculate the rolling window


---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [99]:
# Annualized Sharpe Ratios


In [100]:
# Visualize the sharpe ratios as a bar plot


### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here!

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [101]:
# Reading data from 1st stock


In [102]:
# Reading data from 2nd stock


In [103]:
# Reading data from 3rd stock


In [104]:
# Combine all stocks in a single DataFrame


In [105]:
# Reset Date index


In [106]:
# Reorganize portfolio data by having a column per symbol


In [107]:
# Calculate daily returns

# Drop NAs

# Display sample data


## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [108]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return

# Display sample data


## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [109]:
# Join your returns DataFrame to the original returns DataFrame


In [110]:
# Only compare dates where return data exists for all the stocks (drop NaNs)


## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [111]:
# Calculate the annualized `std`


### Calculate and plot rolling `std` with 21-day window

In [112]:
# Calculate rolling standard deviation

# Plot rolling standard deviation


### Calculate and plot the correlation

In [113]:
# Calculate and plot the correlation


### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [114]:
# Calculate and plot Beta


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [115]:
# Calculate Annualized Sharpe Ratios


In [116]:
# Visualize the sharpe ratios as a bar plot


### How does your portfolio do?

Write your answer here!